In [ ]:
from featsql.featsqlite import *
from featsql.featmysql import *
from featsql.featsnow import *

## Imports

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import mysql.connector

In [ ]:
pd.set_option('display.max_columns', None)

# featsql

> Create features with sql

## Install

```sh
pip install featsql
```

## 1. SQLITE

### Configurando a engine

In [ ]:
#| hide
url_db = "sqlite:///../../data/mydatabase.db" 

engine = create_engine(url_db)

### Visão inicial do público

Primeiro vamos observar o formato da tabela spine

In [ ]:
#| hide
df_spine = pd.read_sql("SELECT * FROM tb_spine", engine)
df_spine.head()

,ID,SAFRA_REF,Target
0,4,2023-02-01,0
1,5,2023-02-01,0
2,6,2023-02-01,0
3,7,2023-02-01,0
4,10,2023-02-01,0


### Visão inicial da tabela de variáveis

A tabela de variáveis contém 4 variáveis, duas sendo numéricas e duas categórica. Perceba que existem mais ID's únicos e datas disponíveis nessa tabela do que na tabela spine, caso que ocorre no dia a dia.

In [ ]:
#| hide
df_data = pd.read_sql("SELECT * FROM tb_feat", engine)
df_data.head()

,ID,SAFRA,FEAT_NUM1,FEAT_NUM2,FEAT_CAT1,FEAT_CAT2
0,1,2023-01-01,-97,-44,A,C
1,2,2023-01-01,89,67,C,B
2,3,2023-01-01,53,24,A,B
3,4,2023-01-01,-40,62,B,C
4,5,2023-01-01,41,62,B,B


### Criação de variáveis numéricas

A função sqlite_create_query_num() cria um texto com a query para a criação de variáveis com as operações soma, mínimo, máximo e média das variáveis listadas em feat_num_lista e com a janela de tempo listada em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]
query_final_num_sqlite = sqlite_create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra)

In [ ]:
#| hide
df_num_sqlite = pd.read_sql(query_final_num_sqlite, engine)
df_num_sqlite.head()

,ID,SAFRA_REF,FEAT_NUM1_SUM_1M,FEAT_NUM1_MIN_1M,FEAT_NUM1_MAX_1M,FEAT_NUM1_AVG_1M,FEAT_NUM2_SUM_1M,FEAT_NUM2_MIN_1M,FEAT_NUM2_MAX_1M,FEAT_NUM2_AVG_1M,FEAT_NUM1_SUM_2M,FEAT_NUM1_MIN_2M,FEAT_NUM1_MAX_2M,FEAT_NUM1_AVG_2M,FEAT_NUM2_SUM_2M,FEAT_NUM2_MIN_2M,FEAT_NUM2_MAX_2M,FEAT_NUM2_AVG_2M,FEAT_NUM1_SUM_3M,FEAT_NUM1_MIN_3M,FEAT_NUM1_MAX_3M,FEAT_NUM1_AVG_3M,FEAT_NUM2_SUM_3M,FEAT_NUM2_MIN_3M,FEAT_NUM2_MAX_3M,FEAT_NUM2_AVG_3M
0,4,2023-02-01,-40.0,-40.0,-40.0,-40.0,62.0,62.0,62.0,62.0,-40,-40,-40,-40.0,62,62,62,62.0,-40,-40,-40,-40.0,62,62,62,62.0
1,5,2023-02-01,41.0,41.0,41.0,41.0,62.0,62.0,62.0,62.0,41,41,41,41.0,62,62,62,62.0,41,41,41,41.0,62,62,62,62.0
2,6,2023-02-01,36.0,36.0,36.0,36.0,63.0,63.0,63.0,63.0,36,36,36,36.0,63,63,63,63.0,36,36,36,36.0,63,63,63,63.0
3,7,2023-02-01,47.0,47.0,47.0,47.0,44.0,44.0,44.0,44.0,47,47,47,47.0,44,44,44,44.0,47,47,47,47.0,44,44,44,44.0
4,10,2023-02-01,29.0,29.0,29.0,29.0,-7.0,-7.0,-7.0,-7.0,29,29,29,29.0,-7,-7,-7,-7.0,29,29,29,29.0,-7,-7,-7,-7.0


Ajustar para não necessariamente criar um dataframe por conta do tamanho

In [ ]:
print(query_final_num_sqlite)


    WITH 
    tb_public AS (
        SELECT 
            *
        FROM tb_spine
    ),
    
        -- Criação de variáveis de janela de 1M
        tb_janela_1M as(
            SELECT 
                tb_public.ID,
                tb_public.SAFRA_REF,
                
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM1 para a janela 1
            SUM(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_SUM_1M,
            MIN(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MIN_1M,
            MAX(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MAX_1M,
            AVG(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_AVG_1M,
            
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM2 para a janela 1
            SUM(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_SUM_1M,
            MIN(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_MIN_1M,
            MAX(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_MAX_1M,
            AVG(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_N

### Criação de variáveis categóricas

A função sqlite_create_query_cat() cria um texto com a query para a criação de variáveis com a moda de cada uma das variáveis listadas  em feat_num_lista na janela de tempo fornecida em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_CAT1', 'FEAT_CAT2']
lista_janela = [1, 3, 6]
query_final_cat_sqlite = sqlite_create_query_cat(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra)

In [ ]:
#| hide
df_cat_sqlite_sqlite = pd.read_sql(query_final_cat_sqlite, engine)
df_cat_sqlite_sqlite.head()

,ID,SAFRA_REF,FEAT_CAT1_MODA_1M,FEAT_CAT2_MODA_1M,FEAT_CAT1_MODA_3M,FEAT_CAT2_MODA_3M,FEAT_CAT1_MODA_6M,FEAT_CAT2_MODA_6M
0,4,2023-02-01,B,C,B,C,B,C
1,5,2023-02-01,B,B,B,B,B,B
2,6,2023-02-01,A,A,A,A,A,A
3,7,2023-02-01,C,B,C,B,C,B
4,10,2023-02-01,A,B,A,B,A,B


In [ ]:
print(query_final_cat_sqlite)


    WITH 
    tb_public as (
        SELECT 
            ID,
            SAFRA_REF
        FROM tb_spine
    ),
    
    tb_janela_FEAT_CAT1_1M as(
        SELECT
            tb_public.ID,
            tb_public.SAFRA_REF,
            tb_feat.FEAT_CAT1,
            COUNT(*) AS frequency_FEAT_CAT1
        FROM tb_public
        LEFT JOIN tb_feat
        ON tb_public.ID = tb_feat.ID
            AND (strftime('%Y-%m-%d', date(tb_feat.SAFRA, '+1 months')) >= tb_public.SAFRA_REF)
            AND (tb_feat.SAFRA < tb_public.SAFRA_REF)
        GROUP BY tb_public.ID, tb_public.SAFRA_REF, tb_feat.FEAT_CAT1
    ),

    tb_row_FEAT_CAT1_1M as (
        SELECT 
            *,    
            ROW_NUMBER() OVER (
                PARTITION BY 
                    ID,
                    SAFRA_REF        
                    ORDER BY frequency_FEAT_CAT1 DESC
            ) as row_num_FEAT_CAT1_1M
        FROM tb_janela_FEAT_CAT1_1M
    ),
    
    tb_moda_FEAT_CAT1_1M AS(
        SELECT
            tb_r

### Criação de variáveis agragadas

In [ ]:
tb_publico = 'tb_spine'
lista_janela = [3, 6]
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT1'
lista_valor_agregador = ['A', 'B']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'

query = sqlite_create_query_agregada(tb_publico, tb_feat, lista_janela, lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agregador)

In [ ]:
print(query)


        WITH
        tb_public as(
        SELECT
            ID,
            SAFRA_REF
        FROM tb_spine
        ),
    
            
    tb_agrupada_FEAT_CAT1_A_3M as(
        SELECT
            tb_public.ID,
            tb_public.SAFRA_REF,

            SUM(COALESCE(tb_feat.FEAT_NUM1,0))  AS SUM_FEAT_NUM1_FEAT_CAT1_A_3M,
            MAX(COALESCE(tb_feat.FEAT_NUM1,0))  AS MAX_FEAT_NUM1_FEAT_CAT1_A_3M,
            MIN(COALESCE(tb_feat.FEAT_NUM1,0))  AS MIN_FEAT_NUM1_FEAT_CAT1_A_3M,
            AVG(COALESCE(tb_feat.FEAT_NUM1,0))  AS AVG_FEAT_NUM1_FEAT_CAT1_A_3M,

            SUM(COALESCE(tb_feat.FEAT_NUM2,0))  AS SUM_FEAT_NUM2_FEAT_CAT1_A_3M,
            MAX(COALESCE(tb_feat.FEAT_NUM2,0))  AS MAX_FEAT_NUM2_FEAT_CAT1_A_3M,
            MIN(COALESCE(tb_feat.FEAT_NUM2,0))  AS MIN_FEAT_NUM2_FEAT_CAT1_A_3M,
            AVG(COALESCE(tb_feat.FEAT_NUM2,0))  AS AVG_FEAT_NUM2_FEAT_CAT1_A_3M
        FROM tb_public
        INNER JOIN tb_feat
            ON tb_public.ID = tb_feat.ID
           

In [ ]:
#| hide
df_sqlite_agregada = pd.read_sql(query, engine)
df_sqlite_agregada.head()

,ID,SAFRA_REF,SUM_FEAT_NUM1_FEAT_CAT1_A_3M,MAX_FEAT_NUM1_FEAT_CAT1_A_3M,MIN_FEAT_NUM1_FEAT_CAT1_A_3M,AVG_FEAT_NUM1_FEAT_CAT1_A_3M,SUM_FEAT_NUM2_FEAT_CAT1_A_3M,MAX_FEAT_NUM2_FEAT_CAT1_A_3M,MIN_FEAT_NUM2_FEAT_CAT1_A_3M,AVG_FEAT_NUM2_FEAT_CAT1_A_3M,SUM_FEAT_NUM1_FEAT_CAT1_A_6M,MAX_FEAT_NUM1_FEAT_CAT1_A_6M,MIN_FEAT_NUM1_FEAT_CAT1_A_6M,AVG_FEAT_NUM1_FEAT_CAT1_A_6M,SUM_FEAT_NUM2_FEAT_CAT1_A_6M,MAX_FEAT_NUM2_FEAT_CAT1_A_6M,MIN_FEAT_NUM2_FEAT_CAT1_A_6M,AVG_FEAT_NUM2_FEAT_CAT1_A_6M,SUM_FEAT_NUM1_FEAT_CAT1_B_3M,MAX_FEAT_NUM1_FEAT_CAT1_B_3M,MIN_FEAT_NUM1_FEAT_CAT1_B_3M,AVG_FEAT_NUM1_FEAT_CAT1_B_3M,SUM_FEAT_NUM2_FEAT_CAT1_B_3M,MAX_FEAT_NUM2_FEAT_CAT1_B_3M,MIN_FEAT_NUM2_FEAT_CAT1_B_3M,AVG_FEAT_NUM2_FEAT_CAT1_B_3M,SUM_FEAT_NUM1_FEAT_CAT1_B_6M,MAX_FEAT_NUM1_FEAT_CAT1_B_6M,MIN_FEAT_NUM1_FEAT_CAT1_B_6M,AVG_FEAT_NUM1_FEAT_CAT1_B_6M,SUM_FEAT_NUM2_FEAT_CAT1_B_6M,MAX_FEAT_NUM2_FEAT_CAT1_B_6M,MIN_FEAT_NUM2_FEAT_CAT1_B_6M,AVG_FEAT_NUM2_FEAT_CAT1_B_6M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-40.0,-40.0,-40.0,-40.0,62.0,62.0,62.0,62.0,-40.0,-40.0,-40.0,-40.0,62.0,62.0,62.0,62.0
1,5,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,41.0,41.0,41.0,62.0,62.0,62.0,62.0,41.0,41.0,41.0,41.0,62.0,62.0,62.0,62.0
2,6,2023-02-01,36.0,36.0,36.0,36.0,63.0,63.0,63.0,63.0,36.0,36.0,36.0,36.0,63.0,63.0,63.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,2023-02-01,29.0,29.0,29.0,29.0,-7.0,-7.0,-7.0,-7.0,29.0,29.0,29.0,29.0,-7.0,-7.0,-7.0,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tb_publico = 'tb_spine'
lista_janela = [3, 6]
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT1'
lista_valor_agragador = ['A', 'B']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'

query = sqlite_create_query_agregada(tb_publico, tb_feat, lista_janela, lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agragador)

In [ ]:
print(query)


        WITH
        tb_public as(
        SELECT
            ID,
            SAFRA_REF
        FROM tb_spine
        ),
    
            
    tb_agrupada_FEAT_CAT1_A_3M as(
        SELECT
            tb_public.ID,
            tb_public.SAFRA_REF,

            SUM(COALESCE(tb_feat.FEAT_NUM1,0))  AS SUM_FEAT_NUM1_FEAT_CAT1_A_3M,
            MAX(COALESCE(tb_feat.FEAT_NUM1,0))  AS MAX_FEAT_NUM1_FEAT_CAT1_A_3M,
            MIN(COALESCE(tb_feat.FEAT_NUM1,0))  AS MIN_FEAT_NUM1_FEAT_CAT1_A_3M,
            AVG(COALESCE(tb_feat.FEAT_NUM1,0))  AS AVG_FEAT_NUM1_FEAT_CAT1_A_3M,

            SUM(COALESCE(tb_feat.FEAT_NUM2,0))  AS SUM_FEAT_NUM2_FEAT_CAT1_A_3M,
            MAX(COALESCE(tb_feat.FEAT_NUM2,0))  AS MAX_FEAT_NUM2_FEAT_CAT1_A_3M,
            MIN(COALESCE(tb_feat.FEAT_NUM2,0))  AS MIN_FEAT_NUM2_FEAT_CAT1_A_3M,
            AVG(COALESCE(tb_feat.FEAT_NUM2,0))  AS AVG_FEAT_NUM2_FEAT_CAT1_A_3M
        FROM tb_public
        INNER JOIN tb_feat
            ON tb_public.ID = tb_feat.ID
           

In [ ]:
#| hide
df_sqlite_agregada = pd.read_sql(query, engine)
df_sqlite_agregada.head()

,ID,SAFRA_REF,SUM_FEAT_NUM1_FEAT_CAT1_A_3M,MAX_FEAT_NUM1_FEAT_CAT1_A_3M,MIN_FEAT_NUM1_FEAT_CAT1_A_3M,AVG_FEAT_NUM1_FEAT_CAT1_A_3M,SUM_FEAT_NUM2_FEAT_CAT1_A_3M,MAX_FEAT_NUM2_FEAT_CAT1_A_3M,MIN_FEAT_NUM2_FEAT_CAT1_A_3M,AVG_FEAT_NUM2_FEAT_CAT1_A_3M,SUM_FEAT_NUM1_FEAT_CAT1_A_6M,MAX_FEAT_NUM1_FEAT_CAT1_A_6M,MIN_FEAT_NUM1_FEAT_CAT1_A_6M,AVG_FEAT_NUM1_FEAT_CAT1_A_6M,SUM_FEAT_NUM2_FEAT_CAT1_A_6M,MAX_FEAT_NUM2_FEAT_CAT1_A_6M,MIN_FEAT_NUM2_FEAT_CAT1_A_6M,AVG_FEAT_NUM2_FEAT_CAT1_A_6M,SUM_FEAT_NUM1_FEAT_CAT1_B_3M,MAX_FEAT_NUM1_FEAT_CAT1_B_3M,MIN_FEAT_NUM1_FEAT_CAT1_B_3M,AVG_FEAT_NUM1_FEAT_CAT1_B_3M,SUM_FEAT_NUM2_FEAT_CAT1_B_3M,MAX_FEAT_NUM2_FEAT_CAT1_B_3M,MIN_FEAT_NUM2_FEAT_CAT1_B_3M,AVG_FEAT_NUM2_FEAT_CAT1_B_3M,SUM_FEAT_NUM1_FEAT_CAT1_B_6M,MAX_FEAT_NUM1_FEAT_CAT1_B_6M,MIN_FEAT_NUM1_FEAT_CAT1_B_6M,AVG_FEAT_NUM1_FEAT_CAT1_B_6M,SUM_FEAT_NUM2_FEAT_CAT1_B_6M,MAX_FEAT_NUM2_FEAT_CAT1_B_6M,MIN_FEAT_NUM2_FEAT_CAT1_B_6M,AVG_FEAT_NUM2_FEAT_CAT1_B_6M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-40.0,-40.0,-40.0,-40.0,62.0,62.0,62.0,62.0,-40.0,-40.0,-40.0,-40.0,62.0,62.0,62.0,62.0
1,5,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,41.0,41.0,41.0,62.0,62.0,62.0,62.0,41.0,41.0,41.0,41.0,62.0,62.0,62.0,62.0
2,6,2023-02-01,36.0,36.0,36.0,36.0,63.0,63.0,63.0,63.0,36.0,36.0,36.0,36.0,63.0,63.0,63.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,2023-02-01,29.0,29.0,29.0,29.0,-7.0,-7.0,-7.0,-7.0,29.0,29.0,29.0,29.0,-7.0,-7.0,-7.0,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. MySQL

### Configurando conexão 

In [ ]:
#| hide
host = "localhost"
user = "sqluser"
password = "password"
database = "mydatabase"

# Conectar ao MySQL
connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

### Visão inicial do público

In [ ]:
#| hide
df_spine = pd.read_sql("SELECT * FROM tb_spine", connection)
df_spine.head()

/tmp/ipykernel_4407/1457666827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_spine = pd.read_sql("SELECT * FROM tb_spine", connection)


,ID,SAFRA_REF,Target
0,4,2023-02-01,1
1,5,2023-02-01,0
2,6,2023-02-01,0
3,7,2023-02-01,0
4,10,2023-02-01,0


### Visão inicial da tabela de variáveis

In [ ]:
#| hide
df_data = pd.read_sql("SELECT * FROM tb_feat", connection)
df_data.head()

/tmp/ipykernel_4407/2401477500.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_data = pd.read_sql("SELECT * FROM tb_feat", connection)


,ID,SAFRA,FEAT_NUM1,FEAT_NUM2,FEAT_CAT1,FEAT_CAT2
0,1,2023-01-01,73,23,B,B
1,3,2023-01-01,15,1,B,B
2,5,2023-01-01,75,71,A,A
3,7,2023-01-01,73,82,B,C
4,9,2023-01-01,61,8,C,B


### Criação de variáveis numéricas

A função mysql_create_query_num() cria um texto com a query para a criação de variáveis com as operações soma, mínimo, máximo e média das variáveis listadas em feat_num_lista e com a janela de tempo listada em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]
query_final_num_mysql = mysql_create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra)

In [ ]:
#| hide
df_num_mysql = pd.read_sql(query_final_num_mysql, connection)
df_num_mysql.head()

/tmp/ipykernel_4407/4045549687.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_num_mysql = pd.read_sql(query_final_num_mysql, connection)


,ID,SAFRA_REF,FEAT_NUM1_SUM_1M,FEAT_NUM1_MIN_1M,FEAT_NUM1_MAX_1M,FEAT_NUM1_AVG_1M,FEAT_NUM2_SUM_1M,FEAT_NUM2_MIN_1M,FEAT_NUM2_MAX_1M,FEAT_NUM2_AVG_1M,FEAT_NUM1_SUM_2M,FEAT_NUM1_MIN_2M,FEAT_NUM1_MAX_2M,FEAT_NUM1_AVG_2M,FEAT_NUM2_SUM_2M,FEAT_NUM2_MIN_2M,FEAT_NUM2_MAX_2M,FEAT_NUM2_AVG_2M,FEAT_NUM1_SUM_3M,FEAT_NUM1_MIN_3M,FEAT_NUM1_MAX_3M,FEAT_NUM1_AVG_3M,FEAT_NUM2_SUM_3M,FEAT_NUM2_MIN_3M,FEAT_NUM2_MAX_3M,FEAT_NUM2_AVG_3M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,2023-02-01,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0
2,6,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0
4,10,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(query_final_num_mysql)


    WITH 
    tb_public AS (
        SELECT 
            *
        FROM tb_spine
    ),
    
        -- Criação de variáveis de janela de 1M
        tb_janela_1M AS (
            SELECT 
                tb_public.ID,
                tb_public.SAFRA_REF,
                
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM1 para a janela 1
            SUM(IFNULL(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_SUM_1M,
            MIN(IFNULL(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MIN_1M,
            MAX(IFNULL(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MAX_1M,
            AVG(IFNULL(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_AVG_1M,
            
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM2 para a janela 1
            SUM(IFNULL(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_SUM_1M,
            MIN(IFNULL(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_MIN_1M,
            MAX(IFNULL(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_MAX_1M,
            AVG(IFNULL(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_AVG_1M
    

### Criação de variáveis categóricas

A função mysql_create_query_cat() cria um texto com a query para a criação de variáveis com a moda de cada uma das variáveis listadas  em feat_num_lista na janela de tempo fornecida em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_CAT1','FEAT_CAT2']
lista_janela = [1,2,3]
query_final_cat_mysql = mysql_create_query_cat(tb_publico, tb_feat, lista_janela, feat_num_lista, id, safra_ref, safra)

In [ ]:
#| hide
df_cat_sqlite_mysql = pd.read_sql(query_final_cat_mysql, connection)
df_cat_sqlite_mysql.head()

/tmp/ipykernel_4407/3384917413.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cat_sqlite_mysql = pd.read_sql(query_final_cat_mysql, connection)


,ID,SAFRA_REF,FEAT_CAT1_MODA_1M,FEAT_CAT2_MODA_1M,FEAT_CAT1_MODA_2M,FEAT_CAT2_MODA_2M,FEAT_CAT1_MODA_3M,FEAT_CAT2_MODA_3M
0,4,2023-02-01,None,None,None,None,None,None
1,5,2023-02-01,A,A,A,A,A,A
2,6,2023-02-01,None,None,None,None,None,None
3,7,2023-02-01,B,C,B,C,B,C
4,10,2023-02-01,None,None,None,None,None,None


In [ ]:
print(query_final_cat_mysql)


    WITH 
    tb_public AS (
        SELECT 
            ID,
            SAFRA_REF
        FROM tb_spine
    ),
    
    tb_janela_FEAT_CAT1_1M AS (
        SELECT
            tb_public.ID,
            tb_public.SAFRA_REF,
            tb_feat.FEAT_CAT1,
            COUNT(*) AS frequency_FEAT_CAT1
        FROM tb_public
        LEFT JOIN tb_feat
        ON tb_public.ID = tb_feat.ID
            AND DATE_ADD(tb_feat.SAFRA, INTERVAL 1 MONTH) >= tb_public.SAFRA_REF
            AND tb_feat.SAFRA < tb_public.SAFRA_REF
        GROUP BY tb_public.ID, tb_public.SAFRA_REF, tb_feat.FEAT_CAT1
    ),

    tb_row_FEAT_CAT1_1M AS (
        SELECT 
            *,    
            ROW_NUMBER() OVER (
                PARTITION BY 
                    ID,
                    SAFRA_REF        
                    ORDER BY frequency_FEAT_CAT1 DESC
            ) AS row_num_FEAT_CAT1_1M
        FROM tb_janela_FEAT_CAT1_1M
    ),
    
    tb_moda_FEAT_CAT1_1M AS (
        SELECT
            tb_row_FEAT_CAT1_1M

### Criação de variáveis agragadas

In [ ]:
tb_publico = 'tb_spine'
lista_janela = [3, 6]
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT1'
lista_valor_agragador = ['A', 'B']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'

query = mysql_create_query_agregada(tb_publico, tb_feat, lista_janela, lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agragador)

In [ ]:
print(query)


        WITH
        tb_public as(
        SELECT
            ID,
            SAFRA_REF
        FROM tb_spine
        ),
    
            
    tb_agrupada_FEAT_CAT1_A_3M as(
        SELECT
            tb_public.ID,
            tb_public.SAFRA_REF,

            SUM(COALESCE(tb_feat.FEAT_NUM1,0))  AS SUM_FEAT_NUM1_FEAT_CAT1_A_3M,
            MAX(COALESCE(tb_feat.FEAT_NUM1,0))  AS MAX_FEAT_NUM1_FEAT_CAT1_A_3M,
            MIN(COALESCE(tb_feat.FEAT_NUM1,0))  AS MIN_FEAT_NUM1_FEAT_CAT1_A_3M,
            AVG(COALESCE(tb_feat.FEAT_NUM1,0))  AS AVG_FEAT_NUM1_FEAT_CAT1_A_3M,

            SUM(COALESCE(tb_feat.FEAT_NUM2,0))  AS SUM_FEAT_NUM2_FEAT_CAT1_A_3M,
            MAX(COALESCE(tb_feat.FEAT_NUM2,0))  AS MAX_FEAT_NUM2_FEAT_CAT1_A_3M,
            MIN(COALESCE(tb_feat.FEAT_NUM2,0))  AS MIN_FEAT_NUM2_FEAT_CAT1_A_3M,
            AVG(COALESCE(tb_feat.FEAT_NUM2,0))  AS AVG_FEAT_NUM2_FEAT_CAT1_A_3M
        FROM tb_public
            INNER JOIN tb_feat 
            ON  tb_public.ID = tb_feat.ID
     

In [ ]:
#| hide
df_mysql_agregada = pd.read_sql(query, connection)
df_mysql_agregada.head()

/tmp/ipykernel_4407/696495608.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mysql_agregada = pd.read_sql(query, connection)


,ID,SAFRA_REF,SUM_FEAT_NUM1_FEAT_CAT1_A_3M,MAX_FEAT_NUM1_FEAT_CAT1_A_3M,MIN_FEAT_NUM1_FEAT_CAT1_A_3M,AVG_FEAT_NUM1_FEAT_CAT1_A_3M,SUM_FEAT_NUM2_FEAT_CAT1_A_3M,MAX_FEAT_NUM2_FEAT_CAT1_A_3M,MIN_FEAT_NUM2_FEAT_CAT1_A_3M,AVG_FEAT_NUM2_FEAT_CAT1_A_3M,SUM_FEAT_NUM1_FEAT_CAT1_A_6M,MAX_FEAT_NUM1_FEAT_CAT1_A_6M,MIN_FEAT_NUM1_FEAT_CAT1_A_6M,AVG_FEAT_NUM1_FEAT_CAT1_A_6M,SUM_FEAT_NUM2_FEAT_CAT1_A_6M,MAX_FEAT_NUM2_FEAT_CAT1_A_6M,MIN_FEAT_NUM2_FEAT_CAT1_A_6M,AVG_FEAT_NUM2_FEAT_CAT1_A_6M,SUM_FEAT_NUM1_FEAT_CAT1_B_3M,MAX_FEAT_NUM1_FEAT_CAT1_B_3M,MIN_FEAT_NUM1_FEAT_CAT1_B_3M,AVG_FEAT_NUM1_FEAT_CAT1_B_3M,SUM_FEAT_NUM2_FEAT_CAT1_B_3M,MAX_FEAT_NUM2_FEAT_CAT1_B_3M,MIN_FEAT_NUM2_FEAT_CAT1_B_3M,AVG_FEAT_NUM2_FEAT_CAT1_B_3M,SUM_FEAT_NUM1_FEAT_CAT1_B_6M,MAX_FEAT_NUM1_FEAT_CAT1_B_6M,MIN_FEAT_NUM1_FEAT_CAT1_B_6M,AVG_FEAT_NUM1_FEAT_CAT1_B_6M,SUM_FEAT_NUM2_FEAT_CAT1_B_6M,MAX_FEAT_NUM2_FEAT_CAT1_B_6M,MIN_FEAT_NUM2_FEAT_CAT1_B_6M,AVG_FEAT_NUM2_FEAT_CAT1_B_6M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,2023-02-01,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,75.0,75.0,75.0,75.0,71.0,71.0,71.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0
4,10,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tb_publico = 'tb_spine'
lista_janela = [3, 6]
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT2'
lista_valor_agragador = ['B', 'C']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'

query = mysql_create_query_agregada(tb_publico, tb_feat, lista_janela, lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agragador)

In [ ]:
print(query)


        WITH
        tb_public as(
        SELECT
            ID,
            SAFRA_REF
        FROM tb_spine
        ),
    
            
    tb_agrupada_FEAT_CAT2_B_3M as(
        SELECT
            tb_public.ID,
            tb_public.SAFRA_REF,

            SUM(COALESCE(tb_feat.FEAT_NUM1,0))  AS SUM_FEAT_NUM1_FEAT_CAT2_B_3M,
            MAX(COALESCE(tb_feat.FEAT_NUM1,0))  AS MAX_FEAT_NUM1_FEAT_CAT2_B_3M,
            MIN(COALESCE(tb_feat.FEAT_NUM1,0))  AS MIN_FEAT_NUM1_FEAT_CAT2_B_3M,
            AVG(COALESCE(tb_feat.FEAT_NUM1,0))  AS AVG_FEAT_NUM1_FEAT_CAT2_B_3M,

            SUM(COALESCE(tb_feat.FEAT_NUM2,0))  AS SUM_FEAT_NUM2_FEAT_CAT2_B_3M,
            MAX(COALESCE(tb_feat.FEAT_NUM2,0))  AS MAX_FEAT_NUM2_FEAT_CAT2_B_3M,
            MIN(COALESCE(tb_feat.FEAT_NUM2,0))  AS MIN_FEAT_NUM2_FEAT_CAT2_B_3M,
            AVG(COALESCE(tb_feat.FEAT_NUM2,0))  AS AVG_FEAT_NUM2_FEAT_CAT2_B_3M
        FROM tb_public
            INNER JOIN tb_feat 
            ON  tb_public.ID = tb_feat.ID
     

In [ ]:
#| hide
df_mysql_agregada = pd.read_sql(query, connection)
df_mysql_agregada.head()

/tmp/ipykernel_4407/696495608.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mysql_agregada = pd.read_sql(query, connection)


,ID,SAFRA_REF,SUM_FEAT_NUM1_FEAT_CAT2_B_3M,MAX_FEAT_NUM1_FEAT_CAT2_B_3M,MIN_FEAT_NUM1_FEAT_CAT2_B_3M,AVG_FEAT_NUM1_FEAT_CAT2_B_3M,SUM_FEAT_NUM2_FEAT_CAT2_B_3M,MAX_FEAT_NUM2_FEAT_CAT2_B_3M,MIN_FEAT_NUM2_FEAT_CAT2_B_3M,AVG_FEAT_NUM2_FEAT_CAT2_B_3M,SUM_FEAT_NUM1_FEAT_CAT2_B_6M,MAX_FEAT_NUM1_FEAT_CAT2_B_6M,MIN_FEAT_NUM1_FEAT_CAT2_B_6M,AVG_FEAT_NUM1_FEAT_CAT2_B_6M,SUM_FEAT_NUM2_FEAT_CAT2_B_6M,MAX_FEAT_NUM2_FEAT_CAT2_B_6M,MIN_FEAT_NUM2_FEAT_CAT2_B_6M,AVG_FEAT_NUM2_FEAT_CAT2_B_6M,SUM_FEAT_NUM1_FEAT_CAT2_C_3M,MAX_FEAT_NUM1_FEAT_CAT2_C_3M,MIN_FEAT_NUM1_FEAT_CAT2_C_3M,AVG_FEAT_NUM1_FEAT_CAT2_C_3M,SUM_FEAT_NUM2_FEAT_CAT2_C_3M,MAX_FEAT_NUM2_FEAT_CAT2_C_3M,MIN_FEAT_NUM2_FEAT_CAT2_C_3M,AVG_FEAT_NUM2_FEAT_CAT2_C_3M,SUM_FEAT_NUM1_FEAT_CAT2_C_6M,MAX_FEAT_NUM1_FEAT_CAT2_C_6M,MIN_FEAT_NUM1_FEAT_CAT2_C_6M,AVG_FEAT_NUM1_FEAT_CAT2_C_6M,SUM_FEAT_NUM2_FEAT_CAT2_C_6M,MAX_FEAT_NUM2_FEAT_CAT2_C_6M,MIN_FEAT_NUM2_FEAT_CAT2_C_6M,AVG_FEAT_NUM2_FEAT_CAT2_C_6M
0,4,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0,73.0,73.0,73.0,73.0,82.0,82.0,82.0,82.0
4,10,2023-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 3. Snowflake

### Criação de variáveis numéricas

A função snow_create_query_num() cria um texto com a query para a criação de variáveis com as operações soma, mínimo, máximo e média das variáveis listadas em feat_num_lista e com a janela de tempo listada em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_NUM1','FEAT_NUM2']
lista_janela = [1,2,3]
query_final_num_snow = snow_create_query_num(tb_publico, tb_feat, lista_janela,feat_num_lista, id, safra_ref, safra)

In [ ]:
print(query_final_num_snow)


    WITH 
    tb_public AS (
        SELECT 
            *
        FROM tb_spine
    ),
    
        -- Criação de variáveis de janela de 1M
        tb_janela_1M AS (
            SELECT 
                tb_public.ID,
                tb_public.SAFRA_REF,
                
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM1 para a janela 1
            SUM(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_SUM_1M,
            MIN(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MIN_1M,
            MAX(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MAX_1M,
            AVG(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_AVG_1M,
            MEDIAN(COALESCE(tb_feat.FEAT_NUM1,0)) AS FEAT_NUM1_MEDIAN_1M,
            
             -- Criação de variáveis numéricas a partir da coluna FEAT_NUM2 para a janela 1
            SUM(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_SUM_1M,
            MIN(COALESCE(tb_feat.FEAT_NUM2,0)) AS FEAT_NUM2_MIN_1M,
            MAX(COALESCE(tb_feat.FEAT_NUM2,0)) AS

### Criação de variáveis categóricas

A função query_final_cat_snow() cria um texto com a query para a criação de variáveis com a moda de cada uma das variáveis listadas  em feat_num_lista na janela de tempo fornecida em lista_janela.

In [ ]:
tb_publico = 'tb_spine'
tb_feat = 'tb_feat'
id = 'ID'
safra_ref = 'SAFRA_REF'
safra = 'SAFRA'
feat_num_lista = ['FEAT_CAT1','FEAT_CAT2']
lista_janela = [1,2,3]
query_final_cat_snow = snow_create_query_cat(tb_publico, tb_feat, lista_janela, feat_num_lista, id, safra_ref, safra)

In [ ]:
print(query_final_cat_snow)


    WITH 
    tb_public AS (
        SELECT 
            ID,
            SAFRA_REF
        FROM tb_spine
    ),
    
    tb_janela_1M AS (
        SELECT
            tb_public.ID,
            tb_public.SAFRA_REF,
            
            MODE(FEAT_CAT1) AS MODE_FEAT_CAT1_1M,
            MODE(FEAT_CAT2) AS MODE_FEAT_CAT2_1M           
            
        FROM tb_public
        LEFT JOIN tb_feat
            ON  tb_public.ID = tb_feat.ID
            AND (DATEADD('month', 1 , TO_DATE(tb_feat.SAFRA, 'YYYY-MM-DD')) >=  tb_public.SAFRA_REF) 
            AND (TO_DATE(tb_feat.SAFRA, 'YYYY-MM-DD') < tb_public.SAFRA_REF)
        GROUP BY tb_public.ID, tb_public.SAFRA_REF
    ),

    tb_janela_2M AS (
        SELECT
            tb_public.ID,
            tb_public.SAFRA_REF,
            
            MODE(FEAT_CAT1) AS MODE_FEAT_CAT1_2M,
            MODE(FEAT_CAT2) AS MODE_FEAT_CAT2_2M           
            
        FROM tb_public
        LEFT JOIN tb_feat
            ON  tb_public.ID = tb_feat.I

### Criação de variáveis agragadas

In [ ]:
lista_feat_num = ['FEAT_NUM1', 'FEAT_NUM2']
feat_cat = 'FEAT_CAT1'
lista_valor_agragador = ['B', 'C']
id = 'ID'
safra_ref = 'SAFRA_REF'
tb_feat = 'tb_feat'
safra = 'SAFRA'
janelas = [1, 2, 3]
tb_publico = 'tb_spine'


print(snow_create_query_agregada(tb_publico, tb_feat, janelas,  lista_feat_num, id, safra_ref, safra, feat_cat, lista_valor_agragador))


WITH
tb_public as(
    SELECT
        ID,
        SAFRA_REF
    FROM tb_spine
),

            
-- Criação de variáveis agrupadas com janela de 1M
tb_agrupada_FEAT_CAT1_B_1M as(
    SELECT
        tb_public.ID,
        tb_public.SAFRA_REF,

        -- Criação de variáveis agrupadas a partir da coluna FEAT_CAT1 e FEAT_NUM1 para a janela 1
        SUM(COALESCE(tb_feat.FEAT_NUM1,0))  AS SUM_FEAT_NUM1_FEAT_CAT1_B_1M,
        MAX(COALESCE(tb_feat.FEAT_NUM1,0))  AS MAX_FEAT_NUM1_FEAT_CAT1_B_1M,
        MIN(COALESCE(tb_feat.FEAT_NUM1,0))  AS MIN_FEAT_NUM1_FEAT_CAT1_B_1M,
        AVG(COALESCE(tb_feat.FEAT_NUM1,0))  AS AVG_FEAT_NUM1_FEAT_CAT1_B_1M,
        MEDIAN(COALESCE(tb_feat.FEAT_NUM1,0))  AS MEDIAN_FEAT_NUM1_FEAT_CAT1_B_1M,

        -- Criação de variáveis agrupadas a partir da coluna FEAT_CAT1 e FEAT_NUM2 para a janela 1
        SUM(COALESCE(tb_feat.FEAT_NUM2,0))  AS SUM_FEAT_NUM2_FEAT_CAT1_B_1M,
        MAX(COALESCE(tb_feat.FEAT_NUM2,0))  AS MAX_FEAT_NUM2_FEAT_CAT1_B_1M,
        MIN(COA